In [ ]:
###  ====================== 前端測試區 - 跟資料庫調用資料 =========================
# coding:utf-8
import pymongo
import json
import jieba
import re
import os  
import sys 

### 建立MongoDB連線
client = pymongo.MongoClient("54.249.57.118", 27017)
db = client['spadeAce']
collection = db.textMiningNews
jieba.set_dictionary('dict.txt.big')
  
### 測試問句分詞結果
def mainQuery(question):

    checkQuestion1 = re.match(r'^[\u4E00-\u9FFFa-zA-Z\?]+$',question )
    ## ~ $ * 問句的開頭  ; 字母 中文 空格 為可以接受的字元
    checkQuestion2 = re.match(r'^[\~\$\*][\u4E00-\u9FFFa-zA-Z\ \-]+$',question )

    ### 確認是否是層別一問題
    if checkQuestion1 != None:
        answer = querySlayer1(question)
        print('layer1',answer)

    ### 確認是否是層別二問題
    elif checkQuestion2 != None and question[0] == '~':
        answer  = querySlayer2(question)
        if answer != 'nodata4u':
            answerChain = ''
            for oneAnswer in answer:  
                answerChain+=oneAnswer['Date']+'\n'
                answerChain+=oneAnswer['Title']+'\n'
                answerChain+=oneAnswer['Link']+'\n'
            print(type(answerChain))
        else:
            print('layer2','nodata4u')
    ### 確認是否是層別三問題
    elif checkQuestion2 != None and question[0] == '$':
        answer  = querySlayer3 (question)
        print('layer3',answer)

    ### 確認是否是層別四問題
    elif checkQuestion2 != None and question[0] == '*':
        answer  = querySlayer4 (question)
        print('layer4',answer)
    
    else:
        print('really '+'nodata4u')
        
    
question = input("Input:" )
mainQuery(question)

In [ ]:
###  ====================== 前端測試區 - 跟資料庫調用資料 =========================

### 問句層別 1 : Simple Coversaton
def querySlayer1(question):
    with open('conversation.json', 'r',encoding='utf-8') as f:
        tmpY = []
        tmpN = []
        conDict = json.load(f)
        cutQuestion = jieba.cut(question, cut_all=False)
        ### 這邊要先作分流，把可以回答的字先裝起來
        for word in cutQuestion:
            if word in conDict.keys():
                tmpY.append(word)
            else:
                tmpN.append(word)
        
        if tmpY:
            ### 這邊是針對問句中的最先抓到的關鍵字回覆，嘿~
            return conDict[tmpY[0]]
        else:
            return 'nodata4u'

###  ====================== 前端測試區 - 跟資料庫調用資料 =========================

### 問句層別 2 : Gamelog & News
def querySlayer2 (question):
    answerList = []
    keyWord = question[1::].lower()
    result = collection.find().sort('Date',-1)
    for oneResult in result:
        if keyWord in oneResult['Tag'] or keyWord in  oneResult['TagByName'] :
            answerList.append(oneResult)
    ### 如果有資料就給前三比，沒有就給default
    if answerList:
        return answerList[0:3]
    else:
        return 'nodata4u'

In [ ]:
###  ====================== 前端測試區 - 跟資料庫調用資料 =========================

### 問句層別 3 : Analysis
### def querySlayer3 (keyWord):

In [ ]:
###  ====================== 前端測試區 - 跟資料庫調用資料 =========================

### 問句層別 4 : Analysis
def querySlayer4 (keyWord):
    print(' ')

In [ ]:
### 讀入問句記錄檔案
f = open('logQuery.txt','r',encoding='utf-8')
info = []

aline = f.readline()

for aline in f:
    info.append(aline)

f.close()


In [ ]:
#### 找出無法滿足使用者的問句
for oneInfo in info:
    try:
        a = oneInfo.split('++++++++++++++++++++')[0]
        b = oneInfo.split('++++++++++++++++++++')[1]
        c = oneInfo.split('++++++++++++++++++++')[2]
    except:
        print('空陣列，OK',oneInfo)
        
ttlQuary = []
perQuary= {}

for oneInfo in info:
    perQuary= {}
    print(oneInfo.split('++++++++++++++++++++')[0].split('SenderID: ')[1])
    print(oneInfo.split('++++++++++++++++++++')[1].split('Question ')[1])
    print(oneInfo.split('++++++++++++++++++++')[2].split('Answer: ')[1])
    perQuary['User'] = oneInfo.split('++++++++++++++++++++')[0].split('SenderID: ')[1]
    perQuary['Question'] = oneInfo.split('++++++++++++++++++++')[1].split('Question ')[1]
    perQuary['Answer'] = oneInfo.split('++++++++++++++++++++')[2].split('Answer: ')[1]
    ttlQuary.append(perQuary)

In [ ]:
### 層別一 需要解決的客戶問題

for pbQuary in ttlQuary:
    if pbQuary['Answer'] == 'nodata4u\n' and  pbQuary['Question'][0] == '1':
        print(pbQuary)

In [ ]:
### 層別二 需要解決的客戶問題

for pbQuary in ttlQuary:
    if pbQuary['Answer'] == 'nodata4u\n' and  pbQuary['Question'][0] == '2':
        print(pbQuary)